In [1]:
import json 

camera_json_path = '/home/shared/frontier_data/fnt_802/2024-12-13-10-36-rec002/processed/output_colmap/output/aligned_evo/cameras.json'
with open(camera_json_path, 'r') as f:
    camera_json = json.load(f)
camera_json

[{'id': 0,
  'img_name': 'images/econ_cam0_image_raw_image_compressed/1734086204.737390472.jpg',
  'width': 1920,
  'height': 1080,
  'position': [9.28840280216616, -4.876705886976523, 0.2734633504084347],
  'rotation': [[-0.6574800679935605, -0.0581168224030482, 0.7512272593196792],
   [-0.7434412871977248, -0.1121060235185284, -0.6593385260855414],
   [0.12253576082505221, -0.9919952996081364, 0.03050103071028547]],
  'fy': 1095.8980818781044,
  'fx': 1095.0453018418207},
 {'id': 1,
  'img_name': 'images/econ_cam0_image_raw_image_compressed/1734086205.042916472.jpg',
  'width': 1920,
  'height': 1080,
  'position': [9.195258179253587, -5.200707117638722, 0.27398294539806944],
  'rotation': [[-0.7714266556473621,
    -0.047411948866245814,
    0.6345494638414163],
   [-0.6277246416981223, -0.10663472395178426, -0.7710971468321549],
   [0.10422422540699668, -0.9931672279895416, 0.05249921985973175]],
  'fy': 1095.8980818781044,
  'fx': 1095.0453018418207},
 {'id': 2,
  'img_name': 'ima

In [6]:
camera_json[0]

{'id': 0,
 'img_name': 'images/econ_cam0_image_raw_image_compressed/1734086204.737390472.jpg',
 'width': 1920,
 'height': 1080,
 'position': [9.28840280216616, -4.876705886976523, 0.2734633504084347],
 'rotation': [[-0.6574800679935605, -0.0581168224030482, 0.7512272593196792],
  [-0.7434412871977248, -0.1121060235185284, -0.6593385260855414],
  [0.12253576082505221, -0.9919952996081364, 0.03050103071028547]],
 'fy': 1095.8980818781044,
 'fx': 1095.0453018418207}

In [13]:
import sys
import os
# sys.path.append(os.path.join(os.getcwd(), '..'))
# from scene.cameras import DummyCamera
import numpy as np
import torch
import math


def getProjectionMatrix(fovX, fovY, primx=0.5, primy=0.5, znear=0.01, zfar=100.0):
    tanHalfFovY = math.tan((fovY / 2))
    tanHalfFovX = math.tan((fovX / 2))

    #primy = 0.5
    #primx = 0.5

    top = tanHalfFovY * znear
    bottom = (1 - primy) * 2 * -top
    top = primy * 2 * top

    right = tanHalfFovX * znear
    left = (1-primx) * 2 * -right
    right = primx * 2 * right

    P = torch.zeros(4, 4)
    z_sign = 1.0
    P[0, 0] = 2.0 * znear / (right - left)
    P[1, 1] = 2.0 * znear / (top - bottom)
    P[0, 2] = (right + left) / (right - left)
    P[1, 2] = (top + bottom) / (top - bottom)
    P[3, 2] = z_sign
    P[2, 2] = z_sign * zfar / (zfar - znear)
    P[2, 3] = -(zfar * znear) / (zfar - znear)
    return P

def get_transformation_matrix(position, rotation):
    T = np.eye(4, 4)
    T[:3, :3] = rotation
    T[:3, 3] = position
    return np.linalg.inv(T.T)    
    
class DummyCamera:
    def __init__(self):
        self.width = None
        self.height = None
        self.image_width = None
        self.image_height = None
        self.fovX = None
        self.fovY = None
        self.transformation_matrix = None
        self.projection_matrix = None
        self.full_proj_transform = None
        self.camera_center = None
        self.cam_info = None
        self.zfar = 100.0 # 100.0
        self.znear = 0.01
        
    def load(self, cam_info):
        self.cam_info = cam_info.copy()
        position = np.array(cam_info["position"])
        rotation = np.array(cam_info["rotation"])
        fx = cam_info["fx"]
        fy = cam_info["fy"]
        self.width = cam_info["width"]
        self.height = cam_info["height"]
        self.image_width = 8 * ((self.width // 4) // 8)
        self.image_height = 8 * ((self.height // 4) // 8)
        self.fovX = 2 * math.atan(self.width / (2 * fx))
        self.fovY = 2 * math.atan(self.height / (2 * fy))
        self.tanfovX = math.tan(self.fovX / 2)
        self.tanfovY = math.tan(self.fovY / 2)
        self.bg = torch.zeros(3).float().cuda()
        transformation_matrix = get_transformation_matrix(position, rotation)
        projection_matrix = getProjectionMatrix(self.fovX, self.fovY).numpy()   
        
        self.transformation_matrix = torch.from_numpy(transformation_matrix).float().cuda()
        self.projection_matrix = torch.from_numpy(projection_matrix).float().cuda()
        self.full_proj_transform = torch.from_numpy(transformation_matrix @ projection_matrix).float().cuda()
        self.camera_center = torch.from_numpy(position).float().cuda()
        return self

    def update(self, position, rotation):
        self.cam_info["position"] = position
        self.cam_info["rotation"] = rotation
        return self.load(self.cam_info)

In [18]:
dummy_camera = DummyCamera()     
dummy_camera.load(camera_json[0])

In [23]:
dummy_camera_list = []

for cam in camera_json:
    dummy_camera = DummyCamera()
    dummy_camera.load(cam)
    dummy_camera_list.append(dummy_camera)

dummy_camera_list[100].world_view_transform

AttributeError: 'DummyCamera' object has no attribute 'world_view_transform'

In [2]:
import json
import open3d as o3d
import numpy as np

# Load JSON data
with open(camera_json_path, "r") as f:
    data = json.load(f)

points = np.array([entry["position"] for entry in data])

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points)

o3d.io.write_point_cloud("cameras_json.pcd", pcd)


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


True